In [42]:
import os
import pretty_midi
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


## Klasik Müzik Verisetini Yükleme

In [49]:
import pretty_midi
import os

def load_midi_files(directory):
    midi_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".midi"):
            midi_path = os.path.join(directory, filename)
            midi_data = pretty_midi.PrettyMIDI(midi_path)
            midi_files.append(midi_data)
    return midi_files

# MAESTRO verilerinin bulunduğu klasörün yolu
midi_directory = 'musicDS2'
midi_files = load_midi_files(midi_directory)

print(f"Loaded {len(midi_files)} MIDI files.")


Loaded 3 MIDI files.


## MIDI Dosyalarını Zaman Dizilerine Dönüştürme

In [50]:
def midi_to_sequence(midi_data):
    notes = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            notes.append((note.start, note.pitch, note.end))
    return notes

# Tüm MIDI dosyalarını işlemeye başla
sequences = []
for midi_data in midi_files:
    sequences.append(midi_to_sequence(midi_data))

print(f"Converted {len(sequences)} MIDI files to note sequences.")


Converted 3 MIDI files to note sequences.


## LSTM Modeli Oluşturma

In [51]:
def create_model(sequence_length, num_classes):
    model = Sequential()
    model.add(LSTM(128, input_shape=(sequence_length, 1), return_sequences=True))  # input_shape değiştirildi
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # num_classes = MIDI notası sayısı
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Modeli oluştur
sequence_length = 100  # örnek bir ardışık notalar uzunluğu
num_classes = 128  # MIDI notalarının toplam sayısı

model = create_model(sequence_length, num_classes)
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_13 (LSTM)                  │ (None, 100, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 231,168 (903.00 KB)

 Trainable params: 231,168 (903.00 KB)

 Non-trainable params: 0 (0.00 B)

## Eğitim İçin Veriyi Hazırlama

In [52]:
def prepare_data(sequences, sequence_length=100, num_classes=128):
    X = []
    y = []
    
    for sequence in sequences:
        for i in range(len(sequence) - sequence_length):
            X.append([note[1] for note in sequence[i:i+sequence_length]])  # sadece pitch bilgisi
            y.append(sequence[i + sequence_length][1])  # bir sonraki notanın pitch'i
    
    X = np.array(X)
    y = np.array(y)
    
    # One-hot encoding
    y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    
    return X, y

# Veriyi hazırlayıp giriş şeklini uygun hale getirme
X, y = prepare_data(sequences, sequence_length)
X = X.reshape((X.shape[0], X.shape[1], 1))  # 3D tensor şekli: (samples, time_steps, features)


## Modeli Eğitme

In [53]:
# Modeli eğit
model.fit(X, y, epochs=50, batch_size=64)


Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 23s 148ms/step - accuracy: 0.0481 - loss: 4.1191
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 148ms/step - accuracy: 0.0913 - loss: 3.2476
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - accuracy: 0.0982 - loss: 3.1565
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 147ms/step - accuracy: 0.1136 - loss: 3.1031
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 145ms/step - accuracy: 0.0963 - loss: 3.0870
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - accuracy: 0.0984 - loss: 3.0852
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 148ms/step - accuracy: 0.1134 - loss: 3.0415
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 145ms/step - accuracy: 0.1093 - loss: 3.0331
Epoch 9/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 145ms/step - accuracy: 0.1146 - loss: 3.0018
Epoch 10/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 144ms/step - accuracy: 0.1232 - loss: 2.9984
Epoch 11/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 145ms/step - accuracy: 0.1188 - loss: 2.9691
Epoch 12/50
131/131

## Modeli Kaydetme

In [54]:
# Modeli kaydet (.h5 formatı)
model.save('music_generator_model.h5')


## Eğitilmiş Modeli Kullanarak Müzik Üretme ve Üretilen Müzikleri MIDI Dosyasına Dönüştürme

In [71]:
def generate_music(model, start_sequence, num_notes, sequence_length=100):
    generated_notes = []
    current_sequence = start_sequence
    
    for _ in range(num_notes):
        prediction = model.predict(current_sequence)
        next_note = np.argmax(prediction)  # tahmin edilen bir sonraki nota
        generated_notes.append(next_note)
        
        # current_sequence'ı güncelle
        current_sequence = np.append(current_sequence[1:], [[next_note]], axis=0)
    
    return generated_notes

# Başlangıç sekansı (örnek: rastgele bir notadan başla)
start_sequence = np.random.randint(0, 128, (sequence_length, 1))

# 250 nota uzunluğunda müzik oluştur
generated_notes = generate_music(model, start_sequence, num_notes=120)
print(f"Generated {len(generated_notes)} notes.")

import datetime  # Zaman için ekledik

def sequence_to_midi(generated_notes, output_file=None):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    current_time = 0

    for note in generated_notes:
        if 0 <= note <= 127:  # Sadece geçerli notaları al
            midi_note = pretty_midi.Note(
                velocity=100,
                pitch=note,
                start=current_time,
                end=current_time + 0.5
            )
            instrument.notes.append(midi_note)
            current_time += 0.5
        else:
            print(f"Atlandı: Geçersiz nota {note}")

    midi.instruments.append(instrument)

    # Eğer dosya ismi verilmezse, otomatik zaman etiketli dosya ismi oluştur
    if output_file is None:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"generated_music_{timestamp}.mid"

    midi.write(output_file)
    print(f"Şarkı kaydedildi: {output_file}")

# Kullanım:
sequence_to_midi(generated_notes)  # output_file vermezsen otomatik isim verir




4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━

In [1]:
from tensorflow.keras.models import load_model

# Modeli yükle
model = load_model('music_generator_model.h5')  # doğru model dosya adını buraya yaz
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


## Keras-TensorFlow Uyumluluk Sorununu Çözme (TensorFlow sürümünün 1.x değil 2.x olması lazım)

In [74]:
import tensorflow as tf

# TensorFlow 2.x'yi tam uyumlu kullanmak için
tf.compat.v1.Session()


## TensorFlow Sürümünü Kontrol Etme

In [80]:
import tensorflow as tf
print(tf.__version__)  # TensorFlow sürümünü kontrol et


2.19.0


## Veri Şekli Kontrolü (X ve y)

In [83]:
print(X.shape)  # X'in boyutlarını yazdır
print(y.shape)  # y'in boyutlarını yazdır


(8375, 100, 1)
(8375, 128)


## Daha Yüksek Accuracy ve Daha Düşük Loss değerleri için Modeli Tekrar Eğitme

In [85]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim verilerini hazırla (X, y)
model.fit(X, y, epochs=50, batch_size=64)


Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 25s 169ms/step - accuracy: 0.3883 - loss: 1.8872
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 32s 242ms/step - accuracy: 0.4147 - loss: 1.8292
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 40s 232ms/step - accuracy: 0.4151 - loss: 1.7774
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step - accuracy: 0.4342 - loss: 1.7596
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.4198 - loss: 1.7745
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.4318 - loss: 1.7274
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step - accuracy: 0.4322 - loss: 1.7375
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.4510 - loss: 1.6953
Epoch 9/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 163ms/step - accuracy: 0.4589 - loss: 1.6616
Epoch 10/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.4646 - loss: 1.6342
Epoch 11/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 156ms/step - accuracy: 0.4784 - loss: 1.6024
Epoch 12/50
131/131

In [87]:
# Modeli kaydet (.h5 formatı)
model.save('music_generator_model2.h5')


## Müzik Üretme ve Üretilen Müziği MIDI Formatında Kaydetme

In [8]:
import numpy as np
from tensorflow.keras.models import load_model

# Modeli yükle
model = load_model('music_generator_model2.h5')  # doğru model dosya adını buraya yaz
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


In [ ]:
def generate_music(model, start_sequence, num_notes, sequence_length=100):
    generated_notes = []
    current_sequence = start_sequence
    
    for _ in range(num_notes):
        prediction = model.predict(current_sequence)
        next_note = np.argmax(prediction)  # tahmin edilen bir sonraki nota
        generated_notes.append(next_note)
        
        # current_sequence'ı güncelle
        current_sequence = np.append(current_sequence[1:], [[next_note]], axis=0)
    
    return generated_notes

sequence_length = 100;
# Başlangıç sekansı (örnek: rastgele bir notadan başla)
start_sequence = np.random.randint(0, 128, (sequence_length, 1))

# 250 nota uzunluğunda müzik oluştur
generated_notes = generate_music(model, start_sequence, num_notes=360)
print(f"Generated {len(generated_notes)} notes.")

import datetime  # Zaman için ekledik

def sequence_to_midi(generated_notes, output_file=None):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    current_time = 0

    for note in generated_notes:
        if 0 <= note <= 127:  # Sadece geçerli notaları al
            midi_note = pretty_midi.Note(
                velocity=100,
                pitch=note,
                start=current_time,
                end=current_time + 0.5
            )
            instrument.notes.append(midi_note)
            current_time += 0.25
        else:
            print(f"Atlandı: Geçersiz nota {note}")

    midi.instruments.append(instrument)

    # Eğer dosya ismi verilmezse, otomatik zaman etiketli dosya ismi oluştur
    if output_file is None:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"generated_music_{timestamp}.mid"

    midi.write(output_file)
    print(f"Şarkı kaydedildi: {output_file}")

# Kullanım:
sequence_to_midi(generated_notes)  # output_file vermezsen otomatik isim verir




NameError: name 'sequence_length' is not defined

## Tekrardan Modeli Eğitme(Fine-tuning)

In [91]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim verilerini hazırla (X, y)
model.fit(X, y, epochs=50, batch_size=64)


Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 25s 167ms/step - accuracy: 0.7031 - loss: 0.8811
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 41s 169ms/step - accuracy: 0.7054 - loss: 0.8868
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 40s 163ms/step - accuracy: 0.7374 - loss: 0.8009
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.7344 - loss: 0.8099
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 30s 231ms/step - accuracy: 0.7346 - loss: 0.8018
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.7416 - loss: 0.7905
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 168ms/step - accuracy: 0.7434 - loss: 0.7691
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.7533 - loss: 0.7471
Epoch 9/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 161ms/step - accuracy: 0.7480 - loss: 0.7594
Epoch 10/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 162ms/step - accuracy: 0.7541 - loss: 0.7417
Epoch 11/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 158ms/step - accuracy: 0.7641 - loss: 0.7311
Epoch 12/50
131/131

In [93]:
# Modeli kaydet (.h5 formatı)
model.save('music_generator_model.h5')


In [ ]:
from tensorflow.keras.models import load_model

# Modeli yükle
model = load_model('music_generator_model.h5')  # doğru model dosya adını buraya yaz
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


## Anlamlı Melodiler İçin Daha Optimize Müzik Üretme Kodu ve Üretilen Müziği MIDI Olarak Kaydetme

In [ ]:
def generate_music(model, start_sequence, num_notes, sequence_length=100):
    generated_notes = []
    current_sequence = start_sequence
    
    for _ in range(num_notes):
        prediction = model.predict(current_sequence)
        next_note = np.argmax(prediction)  # tahmin edilen bir sonraki nota
        generated_notes.append(next_note)
        
        next_note_array = np.array([[[next_note]]], dtype=np.float32)  # (1, 1, 1)
        current_sequence = np.concatenate((current_sequence[:, 1:, :], next_note_array), axis=1)

    
    return generated_notes

def get_random_start_sequence(X):
    index = np.random.randint(0, len(X) - 1)
    start_sequence = X[index]
    start_sequence = start_sequence.reshape(1, start_sequence.shape[0], start_sequence.shape[1])  # (1, 100, 1)
    return start_sequence

# Başlangıç sekansı oluştur
start_sequence = get_random_start_sequence(X)
start_sequence = start_sequence.astype(np.float32)


# 360 nota uzunluğunda müzik oluştur
generated_notes = generate_music(model, start_sequence, num_notes=360)
print(f"Generated {len(generated_notes)} notes.")

import datetime  # Zaman için ekledik

def sequence_to_midi(generated_notes, output_file=None):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    current_time = 0

    for note in generated_notes:
        if 0 <= note <= 127:  # Sadece geçerli notaları al
            midi_note = pretty_midi.Note(
                velocity=100,
                pitch=note,
                start=current_time,
                end=current_time + 0.5
            )
            instrument.notes.append(midi_note)
            current_time += 0.25
        else:
            print(f"Atlandı: Geçersiz nota {note}")

    midi.instruments.append(instrument)

    # Eğer dosya ismi verilmezse, otomatik zaman etiketli dosya ismi oluştur
    if output_file is None:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"generated_music_{timestamp}.mid"

    midi.write(output_file)
    print(f"Şarkı kaydedildi: {output_file}")

# Kullanım:
sequence_to_midi(generated_notes)  # output_file vermezsen otomatik isim verir




Model başarıyla yüklendi.


NameError: name 'X' is not defined